In [1]:
import pandas as pd
import numpy as np

# 1. **데이터 설명**
1. pathway_train, pathway_test, pathway_valid : CCLE에서 pathway score 계산 한 데이터
2. ic_train, ic_test, ic_val : 민감군, 저항군 라벨
3. ccle : 유전자 raw data -> 민감군, 저항군 간 유전자 발현량이 가장 큰 유전자 3개를 genes에 저장
4. tcga: 환자 데이터로 약물 민간/저항 라벨 부착 예정

In [3]:
pathway_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/FORETINIB/train_pathway_score_FORETINIB.csv")
pathway_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/FORETINIB/test_pathway_score_FORETINIB.csv")
pathway_valid = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/FORETINIB/val_pathway_score_FORETINIB.csv")

ic_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/FORETINIB/ic_train_FORETINIB.csv")
ic_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/FORETINIB/ic_test_FORETINIB.csv")
ic_val = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/FORETINIB/ic_val_FORETINIB.csv")

ccle = pd.read_csv("C:/Users/User/비어플 의료/#_filtered_CCLE_gene_expression.csv")
ccle.index = ccle["Unnamed: 0"]




In [6]:
pathway_train.set_index("SampleID", inplace= True)
pathway_test.set_index("SampleID", inplace= True)
pathway_valid.set_index("SampleID", inplace= True)

ic_train.set_index("Unnamed: 0", inplace=True)
ic_test.set_index("Unnamed: 0", inplace=True)
ic_val.set_index("Unnamed: 0", inplace=True)


In [8]:
genes = ["MAP4K1", "WAS", "IL16"  ]

# 2. **전처리**
1. pathway score scale
2. pathway score에 대해 PCA 진행 후 90% 설명력까지 사용
3. train 기준으로 fjt하고 trest, valid, TCGA에는 transform만 해주기
4. ccle, tcga 유전자들도 sacle해주기
5. ic라벨 인코딩`

In [11]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 스케일링 (z-score 정규화가 되어있더라도 PCA 전에 한 번 더 정규화)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pathway_train)
X_test_scaled = scaler.transform(pathway_test)
X_val_scaled = scaler.transform(pathway_valid)
# PCA 수행 (주성분 2개로 축소)
pca = PCA(n_components=0.90)
pathway_pca_train = pca.fit_transform(X_scaled)
pathway_pca_test  = pca.transform(X_test_scaled)
pathway_pca_valid = pca.transform(X_val_scaled)

In [12]:
pca_columns = [f"PC{i+1}" for i in range(pathway_pca_train.shape[1])]
pca_train_df_all = pd.DataFrame(pathway_pca_train, columns=pca_columns, index=pathway_train.index)
pca_train_df_all

pca_valid_df_all = pd.DataFrame(pathway_pca_valid, columns=pca_columns, index=pathway_valid.index)
pca_valid_df_all

pca_test_df_all = pd.DataFrame(pathway_pca_test, columns=pca_columns, index=pathway_test.index)
pca_test_df_all

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45
SampleID,,,,,,,,,,,,,,,,,,,,,
ACH-000800,3.681929,-4.862973,-9.728671,-2.944548,-1.648892,-0.727035,4.476052,0.376491,0.421463,1.193537,...,1.086927,-0.025477,0.117846,-0.555599,0.812338,1.944426,1.107583,1.043530,-1.108781,0.569955
ACH-000678,-4.949398,1.583884,3.927065,-0.644592,2.872685,3.339968,-1.532167,-2.677078,-1.417681,1.910749,...,0.809379,-1.385434,0.212614,-0.474585,0.408651,0.884805,-0.542215,0.191024,0.559767,0.483548
ACH-000138,-4.623480,0.005260,7.097654,0.255884,1.409486,-2.103380,-1.620942,-0.272103,0.682535,-2.378627,...,0.331038,0.236774,-2.013539,0.470652,-0.277983,1.167785,0.220698,-0.951888,0.846735,-0.826809
ACH-000176,-3.328084,4.047785,0.774273,0.959806,0.109598,-3.092675,-1.658121,-1.217636,0.360403,0.413490,...,-0.377550,-0.300714,-0.739391,-0.091952,0.295041,-1.221681,-0.548188,-0.119519,0.139171,0.802792
ACH-000704,10.025654,-1.349258,3.130533,-0.736974,-2.522294,0.723352,-1.432771,-1.719153,0.338472,3.496127,...,-0.173965,0.328948,1.918371,-0.696766,0.328787,0.483656,-0.073900,1.446291,-0.227631,0.631743
ACH-000098,-0.067018,7.106339,0.939537,0.259378,-0.038709,0.419070,-1.625593,0.796999,-0.028383,0.897546,...,0.993094,-0.408732,-0.772405,-1.281492,-0.119609,0.257812,-0.989746,-0.306956,-0.284300,0.382011
ACH-000832,-10.182121,0.782383,3.996618,1.178065,2.334877,1.869998,0.044791,-0.890618,1.655376,0.535773,...,-0.146728,-0.262169,-2.300701,-1.101052,-0.015124,0.594723,0.445789,0.467633,-1.233249,1.106248
ACH-000672,1.453377,0.197466,0.742111,-1.232019,0.222750,-1.092478,-0.564080,0.153612,-0.702907,-2.104713,...,0.689471,-0.997751,1.369255,0.123742,-1.654418,0.723940,-0.102585,-1.705010,-1.159146,-0.710098
ACH-000711,1.434937,-0.630875,1.004345,0.171755,3.483146,2.756863,0.769702,-1.895988,-0.326934,-0.128107,...,1.124675,-0.380319,1.277000,0.539594,-0.438941,0.376913,1.316479,-0.665487,-0.580955,0.433873


In [13]:
ccle = ccle[genes]
ccle_expr_train = ccle.loc[pathway_train.index]
ccle_log_train = np.log2(ccle_expr_train+1)

scaler = StandardScaler()
ccle_scaled_train = pd.DataFrame(scaler.fit_transform(ccle_log_train),
                          columns = ccle_expr_train.columns,
                          index = ccle_expr_train.index)

ccle_expr_valid = ccle.loc[pathway_valid.index]
ccle_log_valid = np.log2(ccle_expr_valid+1)

ccle_scaled_valid = pd.DataFrame(scaler.transform(ccle_log_valid),
                          columns = ccle_expr_valid.columns,
                          index = ccle_expr_valid.index)

ccle_expr_test = ccle.loc[pathway_test.index]
ccle_log_test = np.log2(ccle_expr_test+1)
ccle_scaled_test = pd.DataFrame(scaler.transform(ccle_log_test),
                          columns = ccle_expr_test.columns,
                          index = ccle_expr_test.index)

In [14]:
X_train = pd.concat([pca_train_df_all, ccle_scaled_train], axis = 1)
X_valid = pd.concat([pca_valid_df_all, ccle_scaled_valid], axis = 1)
X_test = pd.concat([pca_test_df_all, ccle_scaled_test], axis = 1)

final_scaler = StandardScaler()
X_train_final = final_scaler.fit_transform(X_train)
X_valid_final = final_scaler.transform(X_valid)
X_test_final = final_scaler.transform(X_test)


In [18]:
# 숫자 라벨로 변환 (sensitive = 1, resistant = 0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(ic_train.iloc[:,0])
y_valid = le.fit_transform(ic_val.iloc[:,0])
y_test = le.fit_transform(ic_test.iloc[:,0])

# **3. 모델링**
1. 로지스틱, 랜덤포레스트, SVM, KNN으로 진행
2. 전처리를 R로 해야했기에 pipeline으로 묶을 수 없어 cross validation이 힘듬
3. 평가지표는 AUC, F1 score 중점으로 사용
4. 수동으로 튜닝 후 성능이 제일 좋은 모델 선택 후 TCGA 라벨 예측

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

In [34]:
## Logistic Regression
lr = LogisticRegression(C=0.001, max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:, 1]

## Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=4, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

## Support Vector Machine (SVM)
svm = SVC(probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
y_proba_svm = svm.predict_proba(X_test)[:, 1]

## K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_proba_knn = knn.predict_proba(X_test)[:, 1]

# ── AdaBoost 추가 ──
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(
    n_estimators=50,
    random_state=42
)
ada.fit(X_train_final, y_train)

y_pred_ada   = ada.predict(X_test_final)
y_proba_ada  = ada.predict_proba(X_test_final)[:, 1]

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [35]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

print("===== Validation Set Evaluation =====")

# Logistic Regression - Validation
y_pred_lr_valid = lr.predict(X_valid)
y_proba_lr_valid = lr.predict_proba(X_valid)[:, 1]
print("\n--- Logistic Regression ---")
print(classification_report(y_valid, y_pred_lr_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_lr_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_lr_valid))

# Random Forest - Validation
y_pred_rf_valid = rf.predict(X_valid)
y_proba_rf_valid = rf.predict_proba(X_valid)[:, 1]
print("\n--- Random Forest ---")
print(classification_report(y_valid, y_pred_rf_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_rf_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_rf_valid))

# Support Vector Machine - Validation
y_pred_svm_valid = svm.predict(X_valid)
y_proba_svm_valid = svm.predict_proba(X_valid)[:, 1]
print("\n--- Support Vector Machine ---")
print(classification_report(y_valid, y_pred_svm_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_svm_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_svm_valid))

# K-Nearest Neighbors - Validation
y_pred_knn_valid = knn.predict(X_valid)
y_proba_knn_valid = knn.predict_proba(X_valid)[:, 1]
print("\n--- K-Nearest Neighbors ---")
print(classification_report(y_valid, y_pred_knn_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_knn_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_knn_valid))
# AdaBoost - Validation
y_pred_ada_valid   = ada.predict(X_valid_final)
y_proba_ada_valid  = ada.predict_proba(X_valid_final)[:, 1]
print("\n--- AdaBoost ---")
print(classification_report(y_valid, y_pred_ada_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_ada_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_ada_valid))

===== Validation Set Evaluation =====

--- Logistic Regression ---
              precision    recall  f1-score   support

   resistant       0.61      1.00      0.76        31
   sensitive       0.00      0.00      0.00        20

    accuracy                           0.61        51
   macro avg       0.30      0.50      0.38        51
weighted avg       0.37      0.61      0.46        51

ROC-AUC: 0.5435483870967742
Confusion matrix:
 [[31  0]
 [20  0]]

--- Random Forest ---
              precision    recall  f1-score   support

   resistant       0.61      1.00      0.76        31
   sensitive       0.00      0.00      0.00        20

    accuracy                           0.61        51
   macro avg       0.30      0.50      0.38        51
weighted avg       0.37      0.61      0.46        51

ROC-AUC: 0.535483870967742
Confusion matrix:
 [[31  0]
 [20  0]]

--- Support Vector Machine ---
              precision    recall  f1-score   support

   resistant       0.61      0.97     

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USER\anaconda3\Lib\site-packages\s

In [38]:
from sklearn.ensemble import VotingClassifier
# ── Soft Voting Ensemble ──
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('svm', svm),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft'
)
voting_clf.fit(X_train_final, y_train)

y_pred_voting  = voting_clf.predict(X_test_final)
y_proba_voting = voting_clf.predict_proba(X_test_final)[:, 1]

print("\n===== Soft Voting Ensemble =====")
print(classification_report(y_test, y_pred_voting, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_voting))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_voting))





===== Soft Voting Ensemble =====
              precision    recall  f1-score   support

   resistant       0.64      0.94      0.76        31
   sensitive       0.67      0.20      0.31        20

    accuracy                           0.65        51
   macro avg       0.66      0.57      0.54        51
weighted avg       0.65      0.65      0.58        51

ROC-AUC: 0.5403225806451614
Confusion matrix:
 [[29  2]
 [16  4]]


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
